In [63]:
import numpy as np 
import pandas as pd 
import re 
import os 
import glob
from pathlib import Path
import collections
import numexpr
import multiprocessing
import ray
#ray.init()
numexpr.set_num_threads(multiprocessing.cpu_count())

4

In [64]:
ROOT_PATH=os.path.abspath(os.curdir)
PATH_DIR_DATA="data/kara"
ALL_DIR_ON_DATA=glob.glob(f'{os.path.join(ROOT_PATH,PATH_DIR_DATA)}/*/')

ROOT_DIR_RAW=list(filter(re.compile(".*raw").match, ALL_DIR_ON_DATA))[0]
ROOT_DIR_BURNET=list(filter(re.compile(".*burnet").match, ALL_DIR_ON_DATA))[0]
ROOT_DIR_ZIPPED=list(filter(re.compile(".*zipped").match, ALL_DIR_ON_DATA))[0]

ALLXML_FILES = list(Path(ROOT_DIR_RAW).glob('**/*.xml'))

In [65]:
ALLXML_FILES[0]
ALLXML_FILES[0].name

'PPM-R15m-NE13143-2018_09_03-19_30.xml'

In [66]:
class Elements:
    XML_ELEMENTS={"NE":".//NE",
              "NEName":".//NE/NEIdentity/NEName",
              "NEType":".//NE/NEIdentity/NEType",
              "Entity":".//Entity",
              "Compliance":".//Entity/Details/Compliance",
              "Counter":".//Entity/Details/Counter"}
    def __init__(self):
         for key, value in Elements.XML_ELEMENTS.items():
             setattr(self, key, value)   

    
    

    

In [67]:
elements = Elements()

In [68]:
NE = pd.read_xml(ALLXML_FILES[0],
                        xpath=elements.NE,attrs_only=True)

NEName = pd.read_xml(ALLXML_FILES[0],
                        xpath=elements.NEName,attrs_only=True)

NEType = pd.read_xml(ALLXML_FILES[0],
                        xpath=elements.NEType)

Entity =pd.read_xml(ALLXML_FILES[0],
                        xpath=elements.Entity,attrs_only=True)

Compliance = pd.read_xml(ALLXML_FILES[0],
                        xpath=elements.Compliance)
Counter = pd.read_xml(ALLXML_FILES[0],
                        xpath=elements.Counter)



In [69]:
NE
NE_ID_VAL=NE.Id.first
NE_ID_VAL

<bound method NDFrame.first of 0    NE-0-13143
Name: Id, dtype: object>

In [70]:
NEName
NEName_val=NEName.longName.first
NEName_val

<bound method NDFrame.first of 0    Emamreza Amol-MA0950/LRAN SPO
Name: longName, dtype: object>

In [71]:
NEType
NEType_val=NEType.NEType.first
NEType_val

<bound method NDFrame.first of 0    SPO1410
Name: NEType, dtype: object>

In [72]:
Entity.drop(columns=["sourceId","shelfId","equipId","emId"])
%time Entity['neId'].map(lambda neId: neId.split("-")[-1])
Entity

CPU times: user 327 µs, sys: 1e+03 ns, total: 328 µs
Wall time: 335 µs


,Id,emId,equipId,neId,shelfId,sourceId
0,EN-0-13143-0-0-3-9-1-0-0-0-0-21-0,EM-0,EQ-0-13143-0-0-3,NE-0-13143,SH-0-13143-0,SO-0-13143-0-0-3-9-1-0-0-0-0
1,EN-0-13143-0-0-4-9-2-0-0-0-0-21-0,EM-0,EQ-0-13143-0-0-4,NE-0-13143,SH-0-13143-0,SO-0-13143-0-0-4-9-2-0-0-0-0
2,EN-0-13143-0-0-4-9-10-0-0-0-0-21-0,EM-0,EQ-0-13143-0-0-4,NE-0-13143,SH-0-13143-0,SO-0-13143-0-0-4-9-10-0-0-0-0
3,EN-0-13143-0-0-4-9-1-0-0-0-0-21-0,EM-0,EQ-0-13143-0-0-4,NE-0-13143,SH-0-13143-0,SO-0-13143-0-0-4-9-1-0-0-0-0
4,EN-0-13143-0-0-4-9-4-0-0-0-0-21-0,EM-0,EQ-0-13143-0-0-4,NE-0-13143,SH-0-13143-0,SO-0-13143-0-0-4-9-4-0-0-0-0
5,EN-0-13143-0-0-4-9-5-0-0-0-0-21-0,EM-0,EQ-0-13143-0-0-4,NE-0-13143,SH-0-13143-0,SO-0-13143-0-0-4-9-5-0-0-0-0
6,EN-0-13143-0-0-4-9-3-0-0-0-0-21-0,EM-0,EQ-0-13143-0-0-4,NE-0-13143,SH-0-13143-0,SO-0-13143-0-0-4-9-3-0-0-0-0
7,EN-0-13143-0-0-4-9-6-0-0-0-0-21-0,EM-0,EQ-0-13143-0-0-4,NE-0-13143,SH-0-13143-0,SO-0-13143-0-0-4-9-6-0-0-0-0
8,EN-0-13143-0-0-4-9-7-0-0-0-0-21-0,EM-0,EQ-0-13143-0-0-4,NE-0-13143,SH-0-13143-0,SO-0-13143-0-0-4-9-7-0-0-0-0
9,EN-0-13143-0-0-4-9-9-0-0-0-0-21-0,EM-0,EQ-0-13143-0-0-4,NE-0-13143,SH-0-13143-0,SO-0-13143-0-0-4-9-9-0-0-0-0


In [73]:
Compliance.drop(columns=["complianceId","complianceType"])

,Compliance
0,Success
1,Success
2,Success
3,Success
4,Success
5,Success
6,Success
7,Success
8,Success
9,Success


In [74]:
Counter

,name,Counter
0,DropEvents,0.000000e+00
1,TotlOctets,2.083125e+11
2,TotPackets,6.359410e+08
3,BroadcPack,5.333000e+03
4,MulticPack,1.340900e+04
...,...,...
199,OvrSizPack,0.000000e+00
200,Fragments,0.000000e+00
201,Jabbers,0.000000e+00
202,Collisions,0.000000e+00


In [75]:
#%time Counter.name.apply(pd.value_counts).to_dict().keys()
%time count_names = list(collections.Counter(Counter.name.to_numpy()))
count_names

CPU times: user 262 µs, sys: 1e+03 ns, total: 263 µs
Wall time: 272 µs


['DropEvents',
 'TotlOctets',
 'TotPackets',
 'BroadcPack',
 'MulticPack',
 'CrcAlgnErr',
 'UndSizPack',
 'OvrSizPack',
 'Fragments',
 'Jabbers',
 'Collisions',
 'Utilizatio']

In [92]:
%time list_filter_Counter = list(map(lambda name: Counter.query(f'name == "{name}"',engine="numexpr"), count_names))

CPU times: user 40.9 ms, sys: 2.09 ms, total: 43 ms
Wall time: 47.3 ms


In [93]:
list_filter_Counter[0]

,name,Counter
0,DropEvents,0.0
12,DropEvents,0.0
24,DropEvents,0.0
36,DropEvents,0.0
48,DropEvents,0.0
60,DropEvents,0.0
72,DropEvents,0.0
84,DropEvents,0.0
96,DropEvents,0.0
108,DropEvents,0.0


In [96]:

# @ray.remote
# def f(df:pd.DataFrame,name:str):
#     return df.rename({"Counter":name},axis=1).drop( ["name"], axis=1)

# def parmap(f, list_df,list_columns):
#     return [f.remote(df,name) for df,name in zip(list_df,list_columns)]

# # Call parmap() on a list consisting of first 5 integers.
# %timeit result_ids = parmap(f, list_filter_Counter,count_names)

# # Get the results
# results = ray.get(result_ids)


%time result_fiter= list(map(lambda name,df :df.rename({"Counter":name},axis=1).drop( ["name"], axis=1).reset_index().drop(['index'],axis=1) ,count_names,list_filter_Counter))

Counter=pd.concat(result_fiter,axis=1)

CPU times: user 41 ms, sys: 2.79 ms, total: 43.7 ms
Wall time: 48.6 ms


In [97]:
Counter

,DropEvents,TotlOctets,TotPackets,BroadcPack,MulticPack,CrcAlgnErr,UndSizPack,OvrSizPack,Fragments,Jabbers,Collisions,Utilizatio
0,0.0,2.083125e+11,635941045.0,5333.0,13409.0,0.0,0.0,0.0,0.0,0.0,0.0,19.6
1,0.0,9.620172e+09,71634941.0,3.0,990.0,0.0,0.0,134804.0,0.0,0.0,0.0,9.8
2,0.0,3.939395e+08,2539525.0,1.0,0.0,0.0,0.0,10044.0,0.0,0.0,0.0,0.3
3,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,8.028546e+09,60882891.0,12.0,990.0,0.0,0.0,156283.0,0.0,0.0,0.0,8.2
5,0.0,1.752692e+09,12597619.0,1.0,0.0,0.0,0.0,32805.0,0.0,0.0,0.0,1.7
6,0.0,8.293324e+09,62675626.0,4.0,994.0,0.0,0.0,65367.0,0.0,0.0,0.0,8.4
7,0.0,1.371270e+09,11957380.0,1.0,0.0,0.0,0.0,380.0,0.0,0.0,0.0,1.4
8,0.0,1.387770e+09,9991279.0,2.0,0.0,0.0,0.0,4851.0,0.0,0.0,0.0,1.4
9,0.0,1.131743e+09,9486460.0,0.0,991.0,0.0,0.0,13417.0,0.0,0.0,0.0,1.1
